# GPT-4 na maturi iz hrvatskog jezika

In [1]:
# Load all the libraries and set up the environment
from dotenv import load_dotenv
import os
from tqdm.auto import tqdm
import openai
import pandas as pd
tqdm.pandas()

load_dotenv("./scripts/local.env")

True

In [31]:
# Set OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY")

In [33]:
# Load the data
df = pd.read_csv("./data/hrvatski_2021_2022.csv")

# Filter out the first 80 rows because rest of the data is from Excel
df = df[:80]
df.head()

,Zadatak,Ulaz,Izlaz,Predviđen odgovor,Točan odgovor,Ispravno,Vrsta pitanja
0,1,Zapuštena\nU jednoj sobi ogromnoga grada\npokr...,D. Pjesma je grafički organizirana.,D,A,FALSE,Književnost (uz polazni tekst)
1,2,Zapuštena\r\nU jednoj sobi ogromnoga grada\r\n...,B. nemogućnost ostvarivanja ispunjenoga života...,B,B,TRUE,Književnost (uz polazni tekst)
2,3,Zapuštena\r\nU jednoj sobi ogromnoga grada\r\n...,A. u čitanju,A,A,TRUE,Književnost (uz polazni tekst)
3,4,"Ivan Gundulić, Osman\r\nSunčanica mlada i lije...",A. Nadao se da će se neki mladić svidjeti njeg...,A,A,TRUE,Književnost (uz polazni tekst)
4,5,"Ivan Gundulić, Osman\r\nSunčanica mlada i lije...",A. Važno mu je da se njegova loza nastavi.,A,A,TRUE,Književnost (uz polazni tekst)


In [35]:
# Create a function that will generate the answers
def generate_answer(prompt):
    try:
        resp = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
                {"role": "system", "content": "Follow the instructions."},
                {"role": "user", "content": prompt},
            ]
        )
        return resp.choices[0]["message"]["content"]
    except Exception as e:
        print(e)
        return None

In [37]:
# Generate the answers
df["answer"] = df["Ulaz"].progress_apply(generate_answer)

  0%|          | 0/80 [00:00<?, ?it/s]

In [40]:
# df["answer"] needs to be only first character of predvidjeni_gpt4 if not None
df.loc[df["answer"].notnull(), "answer"] = df.loc[df["answer"].notnull(), "answer"].apply(lambda x: x[0])

In [41]:
df["answer"].value_counts()

B    33
A    19
D    16
C    12
Name: answer, dtype: int64

In [42]:
# Calculate the accuracy between the correct answers and the generated answers
df["correct"] = df["answer"] == df["Točan odgovor"]

In [43]:
df["correct"].value_counts(normalize=True)

True     0.825
False    0.175
Name: correct, dtype: float64